In [ ]:
import pandas as pd

# --- 1. Configuração ---
arquivo_bruto_entrada = 'raw_data.tsv'
arquivo_organizado_saida = '/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/raw-data/dados-brutos1-1km.tsv'

print(f"Lendo o ficheiro de dados brutos: '{arquivo_bruto_entrada}'")

# --- 2. Leitura e Limpeza Inicial dos Dados ---
try:
    # Lê o DataFrame de dados brutos, que é separado por tabulação
    df_bruto = pd.read_csv(arquivo_bruto_entrada, sep='\t')
    print("Ficheiro bruto carregado com sucesso.")
except FileNotFoundError:
    print(f"ERRO: O ficheiro '{arquivo_bruto_entrada}' não foi encontrado.")
    exit()

# --- 3. Seleção das Colunas Relevantes ---
# Adicionando a coluna do pesquisador à lista de colunas a manter.
colunas_para_manter = [
    'interna_Pesquisador', 'id', 'km', 'interna', 'velocidade', 'umidade_absoluta',
    'umidade_relativa', 'ambiente_seca', 'ambiente_umida', 'ibutg',
    'tpeito', 'pele', 'interna_pele', 'pse', 'ct', 'st', 'fc',
    'pacing', 'vento', 'interna_VO2MAX', 'interna_Massa'
]

# Verifica se todas as colunas necessárias existem no DataFrame
colunas_existentes = [col for col in colunas_para_manter if col in df_bruto.columns]
if len(colunas_existentes) < len(colunas_para_manter):
    colunas_em_falta = set(colunas_para_manter) - set(colunas_existentes)
    print(f"AVISO: As seguintes colunas não foram encontradas e serão ignoradas: {colunas_em_falta}")

df_selecionado = df_bruto[colunas_existentes]
print("Colunas de metadados removidas, mantendo apenas as variáveis de interesse.")

# --- 4. Filtragem dos Dados de Exercício ---
# Removemos as linhas de medições de base (km=0).
df_exercicio = df_selecionado[df_selecionado['km'] > 0].copy()
print(f"Dados de repouso (km=0) removidos. Restaram {len(df_exercicio)} linhas.")

# --- 5. Criação do trial_id e Ordenação Final ---

# Etapa 5.1: Cria uma identidade única combinando o pesquisador e o id do participante.
df_exercicio['id_unico_ensaio'] = df_exercicio['interna_Pesquisador'] + '_' + df_exercicio['id']

# Etapa 5.2: Criar o 'trial_id' numérico a partir desta identidade única.
# pd.factorize() atribui um número inteiro único para cada string única.
df_exercicio['trial_id'] = pd.factorize(df_exercicio['id_unico_ensaio'])[0]

# Etapa 5.3: ORDENAR o DataFrame.
# Primeiro, ordenamos pelo novo 'trial_id' e, depois, por quilómetro ('km').
df_organizado = df_exercicio.sort_values(by=['trial_id', 'km']).reset_index(drop=True)
print("DataFrame reorganizado e ordenado por ensaio único ('trial_id') e 'km'.")

# --- 6. Preparação do DataFrame de Saída ---
# Remove as colunas de identificação originais e intermédias.
df_final = df_organizado.drop(columns=['id', 'interna_Pesquisador', 'id_unico_ensaio'])

colunas_principais = ['trial_id', 'km']
colunas_variaveis = [col for col in df_final.columns if col not in colunas_principais]
ordem_final_colunas = colunas_principais + sorted(colunas_variaveis)
df_final = df_final[ordem_final_colunas]

# --- 7. Salvando o DataFrame Organizado ---
# Este novo DataFrame está pronto para ser usado no notebook de análise e modelagem.
df_final.to_csv(arquivo_organizado_saida, sep=',', index=False, float_format='%.2f')

print(f"\nArquivo de dados organizado salvo com sucesso como: '{arquivo_organizado_saida}'")
print("Este novo DataFrame está pronto para ser usado no pipeline de modelagem.")
print(f"O arquivo final contém {len(df_final)} linhas e {len(df_final.columns)} colunas.")
print("\nVisualização das 15 primeiras linhas do arquivo final gerado (deve mostrar o Ensaio 0):")
print(df_final.head(15))